### Import Relevant Packages

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Load dataset


In [ ]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

# We choose the validation dataset to be 10% of the training dataset
num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# Store number of test samples in a variable
num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Define function to scale data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.float32)
    image /= 255.

    return image, label

scaled_train_val_dataset = mnist_train.map(scale)
test_data = mnist_test.map(scale)

# Shuffle the data
BUFFER_SIZE = 10000

shuffled_train_val_dataset = scaled_train_val_dataset.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_val_dataset.take(num_validation_samples)
train_data = shuffled_train_val_dataset.skip(num_validation_samples)

# Batch the data for GD
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))